In [1]:
VAL = 0.20

In [2]:
import os
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

SEED = 1236
tf.random.set_seed(SEED)

cwd = os.getcwd()

import json
import shutil
import random

# Defining the datasets directory
dataset_dir = os.path.join(cwd, 'MaskDataset')
training_dir = os.path.join(dataset_dir, 'training')
validation_dir = os.path.join(dataset_dir, 'validation')
test_dir = os.path.join(dataset_dir, 'test')

# Create validation directory if it doesn't exist
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)

# Loading the classes of each image into the memory
train_classes_json_file_name = 'train_gt.json'
train_classes_json_directory = os.path.join(dataset_dir, train_classes_json_file_name)

data = {}

with open(train_classes_json_directory) as json_file:
    data = json.load(json_file)


# Creating folder for each class of image for training and validation datasets
classes = set(data.values())
print(classes)

for class_label in classes:
    class_training_dir = os.path.join(training_dir, str(class_label))
    class_validation_dir = os.path.join(validation_dir, str(class_label))
    if not os.path.exists(class_training_dir):
        os.makedirs(class_training_dir)
    if not os.path.exists(class_validation_dir):
        os.makedirs(class_validation_dir)

# Assigning images to each training folder/class, avoiding to have the same image two times in the same folder
for entry in os.scandir(training_dir):
    if(entry.is_file()):
        file_destination = os.path.join(training_dir, str(data[entry.name]), entry.name)
        if not os.path.isfile(file_destination):
            shutil.copy(entry.path, file_destination)
    
# Choosing random images to be into the validation folders, being able to repeat without cloning images
validation_rate = VAL

for class_label in classes:
    class_training_dir = os.path.join(training_dir, str(class_label))
    class_validation_dir = os.path.join(validation_dir, str(class_label))
    
    for old_entry in os.scandir(class_validation_dir):
        os.remove(old_entry.path)
    
    training_entries = list(os.scandir(class_training_dir))
    validation_size = round(len(training_entries)*validation_rate)
    
    for validation_entry in random.sample(training_entries, validation_size):
        destination = os.path.join(class_validation_dir, validation_entry.name)
        os.rename(validation_entry.path, destination)

{0, 1, 2}


# Data augmentation

In [3]:
apply_data_augmentation = True

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=10,
        height_shift_range=10,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='constant',
        cval=0,
        rescale=1/255.
    )
else:
    train_data_gen = ImageDataGenerator(rescale=1/255.)

valid_data_gen = ImageDataGenerator(rescale=1/255.)
# test_data_gen = ImageDataGenerator(rescale=1/255.)

bs = 16

train_gen = train_data_gen.flow_from_directory(
    training_dir,
    batch_size=bs,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

valid_gen = valid_data_gen.flow_from_directory(
    validation_dir,
    batch_size=bs,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

# test_gen = test_data_gen.flow_from_directory(
#     test_dir,
#     batch_size=bs,
#     class_mode='categorical',
#     shuffle=True,
#     seed=SEED
# )

img_h = 256
img_w = 256

num_classes = len(classes)

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_gen,
    output_types=(tf.float16, tf.float16),
    output_shapes=([None, 256, 256, 3], [None, num_classes])
)

train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(
    lambda: valid_gen,
    output_types=(tf.float16, tf.float16),
    output_shapes=([None, 256, 256, 3], [None, num_classes])
)

valid_dataset = valid_dataset.repeat()

# test_dataset = tf.data.Dataset.from_generator(
#     lambda: test_gen,
#     output_types=(tf.float16, tf.float16),
#     output_shapes=([None, 256, 256, 3], [None, num_classes])
# )

# test_dataset = test_dataset.repeat()

# encode text category labels 
from sklearn.preprocessing import LabelEncoder 

from keras.utils import to_categorical
train_labels = to_categorical(class_label)

Found 4492 images belonging to 3 classes.
Found 1122 images belonging to 3 classes.
Using TensorFlow backend.


# Building the Network

In [4]:
model = tf.keras.Sequential()

In [5]:
start_f = 32
depth = 5

model = tf.keras.Sequential()
for i in range(depth):
    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape = [None]

    # Convolutional part
    model.add(
        tf.keras.layers.Conv2D(
            filters=start_f,
            kernel_size=(4, 4),
            strides=(1, 1),
            padding="same",
            input_shape=input_shape
    ))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    start_f *= 2

# Fully connected part
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      1568      
_________________________________________________________________
re_lu (ReLU)                 (None, 256, 256, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      32832     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 128, 128, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 64)     

In [7]:
loss = tf.keras.losses.CategoricalCrossentropy()

lr = 1e-4

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [8]:
%load_ext tensorboard
%tensorboard --logdir 'D:\Polimi\Lectures\3semester\artificial-neural-networks-and-deep-learning\Homeworks\1st\classification_experiments\'

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2020-11-22 00:04:42.993952: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_101.dll
Traceback (most recent call last):
  File "C:\Users\enric\anaconda3\envs\tensorflow_gpuenv\Scripts\tensorboard-script.py", line 9, in <module>
    sys.exit(run_main())
  File "C:\Users\enric\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\tensorboard\main.py", line 65, in run_main
    default.get_plugins() + default.get_dynamic_plugins(),
  File "C:\Users\enric\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\tensorboard\default.py", line 125, in get_dynamic_plugins
    "tensorboard_plugins"
  File "C:\Users\enric\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\tensorboard\default.py", line 124, in <listcomp>
    for entry_point in pkg_resources.iter_entry_points(
  File "C:\Users\enric\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\pkg_resources\__init__.py",

In [9]:
exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(ckpt_dir, 'cp_.ckpt'), #'cp_{epoch:02d}.ckpt'
    save_weights_only=True # False to save the model directly
)
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)

# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir=tb_dir,
    profile_batch=0,
    histogram_freq=1 # if 1 shows weights histograms
)
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)

In [10]:
history = model.fit(
    x=train_dataset,
    epochs=45,
    steps_per_epoch=len(train_gen),
    validation_data=valid_dataset,
    validation_steps=len(valid_gen),
    callbacks=callbacks
)




Train for 281 steps, validate for 71 steps
Epoch 1/35
281/281 [==============================] - 197s 702ms/step - loss: 1.1010 - accuracy: 0.3497 - val_loss: 1.0942 - val_accuracy: 0.4358
Epoch 2/35
281/281 [==============================] - 190s 678ms/step - loss: 1.0853 - accuracy: 0.3956 - val_loss: 1.0791 - val_accuracy: 0.4225
Epoch 3/35
281/281 [==============================] - 192s 684ms/step - loss: 1.0390 - accuracy: 0.4553 - val_loss: 0.9935 - val_accuracy: 0.5089
Epoch 4/35
281/281 [==============================] - 190s 677ms/step - loss: 0.9174 - accuracy: 0.5470 - val_loss: 0.9190 - val_accuracy: 0.5918
Epoch 5/35
281/281 [==============================] - 191s 678ms/step - loss: 0.8333 - accuracy: 0.5906 - val_loss: 0.8344 - val_accuracy: 0.6257
Epoch 6/35
281/281 [==============================] - 190s 676ms/step - loss: 0.7596 - accuracy: 0.6327 - val_loss: 0.8136 - val_accuracy: 0.5900
Epoch 7/35
281/281 [==============================] - 190s 676ms/step - loss: 0.7

In [11]:
# Gráficas
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 30), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, 30), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 30), history.history["acc"], label="train_acc")
plt.plot(np.arange(0, 30), history.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

model.save('1st model save')

ValueError: x and y must have same first dimension, but have shapes (30,) and (35,)

# Read the CSV 

In [9]:
import pandas as pd
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in os.listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1

create_csv(results,dataset_dir)

Found 450 validated image filenames belonging to 1 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
57/57 [==============================] - 15s 262ms/step


## Using ResNet50

In [12]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(img_h,img_w,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

                      
__________________________________________________________________________________________________
res4a_branch2a (Conv2D)         (None, 16, 16, 256)  131328      activation_22[0][0]              
__________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizati (None, 16, 16, 256)  1024        res4a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_23 (Activation)      (None, 16, 16, 256)  0           bn4a_branch2a[0][0]              
__________________________________________________________________________________________________
res4a_branch2b (Conv2D)         (None, 16, 16, 256)  590080      activation_23[0][0]              
__________________________________________________________________________________________________
bn4a_branch2b (BatchNormalizati (None, 16, 16, 256)  1024        res4a_branch2b[0][0] 

In [13]:
start_f = 16
depth = 5

model = tf.keras.Sequential()
for i in range(depth):
    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape = [None]

loss = tf.keras.losses.CategoricalCrossentropy()

lr = 1e-4

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [14]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model_tf = Sequential()
model_tf.add(restnet)
model_tf.add(Dense(256, activation='relu', input_dim=input_shape))
model_tf.add(Dropout(0.3))
model_tf.add(Dense(256, activation='relu'))
model_tf.add(Dropout(0.3))
model_tf.add(Dense(1, activation='sigmoid'))
model_tf.compile(loss=loss,
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])
model_tf.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 131072)            23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33554688  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 57,208,449
Trainable params: 33,620,737
Non-trainable params: 23,587,712
_________________________________

In [7]:
history = model_tf.fit_generator(train_gen, 
                              steps_per_epoch=100, 
                              epochs=100,
                              validation_data=valid_gen, 
                              validation_steps=50,
                              verbose=1)

Epoch 1/100
100/100 [==============================] - 118s 1s/step - loss: 1.1921e-07 - accuracy: 0.4244 - val_loss: 1.1921e-07 - val_accuracy: 0.3333
Epoch 2/100
100/100 [==============================] - 114s 1s/step - loss: 1.1921e-07 - accuracy: 0.4332 - val_loss: 1.1921e-07 - val_accuracy: 0.3333
Epoch 3/100
100/100 [==============================] - 115s 1s/step - loss: 1.1921e-07 - accuracy: 0.4300 - val_loss: 1.1921e-07 - val_accuracy: 0.3333
Epoch 4/100
100/100 [==============================] - 109s 1s/step - loss: 1.1921e-07 - accuracy: 0.4225 - val_loss: 1.1921e-07 - val_accuracy: 0.3333
Epoch 5/100
 66/100 [==================>...........] - ETA: 22s - loss: 1.1921e-07 - accuracy: 0.4198

KeyboardInterrupt: 

In [ ]:
# Gráficas
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 30), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 30), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 30), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 30), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

In [ ]:
import pandas as pd
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in os.listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1

create_csv(results,dataset_dir)